In [33]:
import spacy
import pdfminer
import re
import os
import pandas as pd

In [34]:
import pdff2txt

In [35]:
# converting pdf to txt, 
def convert_pdf(f):
  output_filename = os.path.basename(os.path.splitext(f)[0]) + ".txt"
  output_filepath = os.path.join("output/txt/", output_filename)
  pdff2txt.main(args=[f, "--outfile", output_filepath])
  print(output_filepath + " saved successfully!!!")
  return open(output_filepath).read()

In [36]:
# load the language model
nlp = spacy.load("en_core_web_sm")

In [37]:
result_dict = {"name": [], "phone": [], "email":[], "skills": []}
names = []
phones = []
emails = []
skills = []

In [38]:
#
def parse_content(text):
  skillset = re.compile("python|java|sql|hadoop|tableu")
  phone_num = re.compile(
    "(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})"
  )
  doc = nlp(text)
  name = [entity.text for entity in doc.ents if entity.label_ == "PERSON"][0]
  print(name)
  email = [word for word in doc if word.like_email == True][0]
  print(email)
  phone = str(re.findall(phone_num, text.lower()))
  skills_list = re.findall(skillset, text.lower())
  unique_skills_list = str(set(skills_list))
  names.append(name)
  emails.append(email)
  phones.append(phone)
  skills.append(unique_skills_list)
  print("Extraction completed succesfully!!!")

In [39]:
for file in os.listdir("resumes/"):
  if file.endswith("pdf"):
    txt = convert_pdf(os.path.join("resumes/", file))
    print("\nReading..." + file)
    parse_content(txt)

output/txt/Alisson ParkerCV.txt saved successfully!!!

Reading...Alisson ParkerCV.pdf
Alisson Parker-Wright                                                                                                 
alli1414parks@mail.com
Extraction completed succesfully!!!
output/txt/AshleyMilesCV.txt saved successfully!!!

Reading...AshleyMilesCV.pdf
Ashley Miles
ashleymiles@memail.com
Extraction completed succesfully!!!
output/txt/John DominicCV.txt saved successfully!!!

Reading...John DominicCV.pdf
John Dominic
johndominic@mail.com
Extraction completed succesfully!!!


In [40]:
result_dict["name"] = names
result_dict["phone"] = phones
result_dict["email"] = emails
result_dict["skills"] = skills

In [41]:
result_df = pd.DataFrame(result_dict)
result_df

,name,phone,email,skills
0,Alisson Parker-Wright ...,['8569878511'],alli1414parks@mail.com,"{'python', 'java'}"
1,Ashley Miles,['6592251422'],ashleymiles@memail.com,{'sql'}
2,John Dominic,['7877756411'],johndominic@mail.com,"{'hadoop', 'python', 'java'}"


In [43]:
result_df.to_csv("output/csv/parsed_resumes.csv")

In [44]:
result_df[result_df.skills == str("sql")]

,name,phone,email,skills
